#### NGS data analysis

In [3]:
import math
import numpy as np
import pandas as pd
import itertools
import statistics
from datetime import datetime

# Get current date
dt = datetime.now()
# Format the date
formatted_date = dt.strftime("%Y%m%d")

# Define targeted positions in the region 1 of P450_D12
positions = ['A50', 'R51', 'L54', 'S55', 'L59', 'T61', 'D62', 'S63', 'F71', 'R85', 'T86', 
             'A87', 'L88', 'L89', 'G90', 'E91', 'D93', 'V95', 'H97', 'Q99', 'R101', 'M102', 
             'L103', 'I104', 'P105', 'S106', 'T108', 'L109', 'R111', 'T112', 'L115', 'I119']

def read_screening(screening_file, positions):
    
    # List of all possible amino acids
    amino_acids = ['A', 'C', 'D', 'E', 'F', 
                   'G', 'H', 'I', 'K', 'L', 
                   'M', 'N', 'P', 'Q', 'R', 
                   'S', 'T', 'V', 'W', 'Y']

    # Load NGS data and experimental output
    df = pd.read_excel("20230706_SSM_rd2_onT86L_reanalyzed.xlsx")

    # Read sequencing and fitness data
    mutations = df['VariantCombo'].tolist()
    fitness = df['SSM_rd2_screening'].tolist()
    
    # Initialize the list of all combinations
    all_combinations = []

    # Iterate over the positions
    for pos in positions:
        # Create a copy of the amino acids list without the original amino acid
        AA = [aa for aa in amino_acids if aa != pos[0]]

        # Generate combinations of positions and amino acids
        combinations = list(itertools.product([pos[1:]], AA))  # Changed pos to [pos[1:]], to take the entire string as a single entity

        # Create strings of format 'A50D', 'A50E', etc.
        formatted_combinations = ['{}{}'.format(pos[0], str(p)+aa) for p, aa in combinations]  # Added pos[0] at the beginning to keep the original amino acid in the mutation

        # Add to the list of all combinations
        all_combinations.extend(formatted_combinations)
    
    # Initialize an empty dictionary to store the data
    seq_to_fitness = {}

    # Template fitness
    template_fit = []

    # Iterate over the mutations and corresponding fitness outputs
    for mutation, fit in zip(mutations, fitness):
        # Exclude mutations with certain criteria
        if mutation == "#DEAD#" or math.isnan(fit) or mutation[-1] == "*":
            # print(mutation, fit)
            continue

        # Handle "#PARENT#" mutations
        if mutation == "#PARENT#" and not math.isnan(fit) and fit >= 0.5:
            template_fit.append(fit)
            continue

        # Add mutation to seq_to_fitness if it's not already there and doesn't meet the exclusion criteria
        if mutation not in seq_to_fitness and not math.isnan(fit) and len(mutation) <= 5 and mutation[0] != mutation[-1] and mutation in all_combinations:
            seq_to_fitness[mutation] = [fit]

        elif mutation in seq_to_fitness:
            seq_to_fitness[mutation].append(fit)
    
    # Add the fitness score of the template into the dict
    seq_to_fitness['#PARENT#'] = template_fit
    
    print('Template numbers: ', len(seq_to_fitness['#PARENT#']))
    print('Fitness score of the template: ', round(np.mean(seq_to_fitness['#PARENT#']), 2))
    print('Qualified datapoints: ', sum(len(values) for values in seq_to_fitness.values()))
    print('Mutation numbers: ', len(seq_to_fitness.keys())-1)
    print('Template ratio in total qualified datapoints: ', 
          round(len(seq_to_fitness['#PARENT#'])/sum(len(values) for values in seq_to_fitness.values()), 2))
    print('Library coverage: ', len(seq_to_fitness.keys())/(len(positions)*19))
    
    # Save qualified outputs into an excel file
    mutations = list(seq_to_fitness.keys())
    fitness_score = [np.mean(fit) for fit in seq_to_fitness.values()]
    fitness_stdev = [statistics.stdev(values) if len(values) > 1 else float('nan') for values in seq_to_fitness.values()]
    
    # Creating a dataframe
    df_output = pd.DataFrame({
    'Mutations': mutations,
    'Fitness Score': fitness_score,
    'Fitness Standard Deviation': fitness_stdev
    })
    
    # Saving the dataframe to an Excel file
    df_output.to_excel(f'{formatted_date}_seq_to_fitness_reanalyzed.xlsx', index=False)
          
    return seq_to_fitness

screening_file = "20230706_SSM_rd2_onT86L.xlsx"
seq_to_fitness = read_screening(screening_file, positions)

Template numbers:  56
Fitness score of the template:  1.02
Qualified datapoints:  483
Mutation numbers:  253
Template ratio in total qualified datapoints:  0.12
Library coverage:  0.41776315789473684


#### Prepare the input for ESM-1v

In [ ]:
import numpy as np
import pandas as pd
from Bio import SeqIO

# read the protein sequence
for record in SeqIO.parse('P450_D12_T86L.fasta', "fasta"):
    protein_seq = list(record.seq)

# Define the list of all 20 amino acids
all_amino_acids = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

# Initialize a list to store the mutations
mutations = []

# Loop through each position in the sequence
for n in range(len(protein_seq)):
    # Loop through each possible amino acid
    for aa in all_amino_acids:
        # Don't record mutations that are identical to the original
        if protein_seq[n] != aa:
            # Record the mutation
            mut = protein_seq[n] + str(n) + aa
            mutations.append(mut)

df = pd.DataFrame(mutations, columns=['mutant']) 
df.to_csv('P450_D12_T86L_wholelength.csv', index=True)

#### Correlation between two models and models and screening data

In [25]:
import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.stats import pearsonr

def extract_position(s):
    # Extract the numeric part of the string (position)
    match = re.search(r'\d+', s)
    if match:
        return int(match.group())  # Return the numeric part as an integer
    return np.nan  # If no number is found, return NaN

def increment_number_in_string(s):
    # Find the number using regex
    match = re.search(r'\d+', s)
    if match:
        # Increment the number by 1
        number = str(int(match.group()) + 1)
        # Replace the original number with the new number
        s = s.replace(match.group(), number)
    return s

# Model files
EV_file = "P450_D12_T86L_b0.7_single_mutant_matrix.csv"
ESM_file = "P450_D12_T86L_wholelength_labeled.csv"

# Read EVcouplings data
EV_df = pd.read_csv(EV_file)
epistatic = dict(zip(EV_df['mutant'].tolist(), EV_df['prediction_epistatic'].tolist())) 
independent = dict(zip(EV_df['mutant'].tolist(), EV_df['prediction_independent'].tolist())) 

# Read ESM-1v data
ESM_df = pd.read_csv(ESM_file)
ESM_df = ESM_df.drop(ESM_df.columns[[0, 1]], axis=1)

# Calculate the mean of columns 2 through 6 and store it in a new column 'average'
ESM_df['effect'] = ESM_df.iloc[:, 2:7].mean(axis=1)
    
# Apply the function to the 'mutation' column
ESM_df['mutant'] = ESM_df['mutant'].apply(increment_number_in_string)
    
# Get the prediction from ESM-1v
effect = dict(zip(ESM_df['mutant'].tolist(), ESM_df['effect'].tolist())) 

# Organize data
labels = []
exp_results = []
EV_epistatic = []
EV_independent = []
ESM = []
for key in seq_to_fitness.keys():
    if key != "#PARENT#":
        labels.append(key)
        exp_results.append(np.mean(seq_to_fitness[key]))
        EV_epistatic.append(epistatic[key])
        EV_independent.append(independent[key])
        ESM.append(effect[key])

# Perform linear regression
slope_epi, intercept_epi, r_value_epi, _, _ = stats.linregress(EV_epistatic, exp_results)
slope_inde, intercept_inde, r_value_inde, _, _ = stats.linregress(EV_independent, exp_results)
slope_ESM, intercept_ESM, r_value_ESM, _, _ = stats.linregress(ESM, exp_results)
slope_epi_ESM, intercept_epi_ESM, r_value_epi_ESM, _, _ = stats.linregress(EV_epistatic, ESM)

# Create a subplot with 2 rows and 2 columns
fig = make_subplots(rows=2, cols=2, subplot_titles=("ESM-1v vs. EVcouplings (epistatic)", 
                                                    "Experimental vs. EVcouplings (epistatic)", 
                                                    "Experimental vs. EVcouplings (independent)", 
                                                    "Experimental vs. ESM-1v"),
                    horizontal_spacing=0.15, vertical_spacing=0.15)

# (1, 1) Add scatter plot for correlation between EV Epistatic and ESM
fig.add_trace(go.Scatter(x=EV_epistatic, y=ESM, mode='markers', text=labels, name='Epistatic vs ESM', marker_color='rgb(126, 140, 173)'),
                row=1, col=1)

# Add line plot for correlation between EV Epistatic and ESM
fig.add_trace(go.Scatter(x=EV_epistatic, y=[slope_epi_ESM*x+intercept_epi_ESM for x in EV_epistatic], mode='lines', name='Epistatic vs ESM Line', line=dict(color='black', dash='dash')),
                row=1, col=1)

# (1, 2) Add scatter plot for Experimental vs Epistatic
fig.add_trace(go.Scatter(x=EV_epistatic, y=exp_results, mode='markers', text=labels, name='Epistatic', marker_color='rgb(165, 194, 205)'),
                row=1, col=2)

# Add line plot for Epistatic
fig.add_trace(go.Scatter(x=EV_epistatic, y=[slope_epi*x+intercept_epi for x in EV_epistatic], mode='lines', name='Epistatic Line', line=dict(color='black', dash='dash')),
                row=1, col=2)

# (2, 1) Add scatter plot for Experimental vs Independent
fig.add_trace(go.Scatter(x=EV_independent, y=exp_results, mode='markers', text=labels, name='Independent', marker_color='rgb(196, 201, 194)'),
                row=2, col=1)

# Add line plot for Independent
fig.add_trace(go.Scatter(x=EV_independent, y=[slope_inde*x+intercept_inde for x in EV_independent], mode='lines', name='Independent Line', line=dict(color='black', dash='dash')),
                row=2, col=1)

# (2, 2) Add scatter plot for Experimental vs ESM
fig.add_trace(go.Scatter(x=ESM, y=exp_results, mode='markers', text=labels, name='ESM', marker_color='rgb(155, 142, 149)'),
                row=2, col=2)

# Add line plot for ESM
fig.add_trace(go.Scatter(x=ESM, y=[slope_ESM*x+intercept_ESM for x in ESM], mode='lines', name='ESM Line', line=dict(color='black', dash='dash')),
                row=2, col=2)

# Update x-axis and y-axis properties
fig.update_xaxes(title_text="EVcouplings scores (epistatic)", row=1, col=1, showgrid=True, ticks="outside", tickwidth=2, ticklen=5)
fig.update_yaxes(title_text="ESM-1v scores", row=1, col=1, showgrid=True, ticks="outside", tickwidth=2, ticklen=5)
fig.update_xaxes(title_text="EVcouplings scores (epistatic)", row=1, col=2, showgrid=True, ticks="outside", tickwidth=2, ticklen=5)
fig.update_yaxes(title_text="Fold improvement in activity", row=1, col=2, showgrid=True, ticks="outside", tickwidth=2, ticklen=5)
fig.update_xaxes(title_text="EVcouplings socres (independent)", row=2, col=1, showgrid=True, ticks="outside", tickwidth=2, ticklen=5)
fig.update_yaxes(title_text="Fold improvement in activity", row=2, col=1, showgrid=True, ticks="outside", tickwidth=2, ticklen=5)
fig.update_xaxes(title_text="ESM-1v scores", row=2, col=2, showgrid=True, ticks="outside", tickwidth=2, ticklen=5)
fig.update_yaxes(title_text="Fold improvement in activity", row=2, col=2, showgrid=True, ticks="outside", tickwidth=2, ticklen=5)
    
fig.update_layout(
    plot_bgcolor='white',
    xaxis1=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    yaxis1=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    xaxis2=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    yaxis2=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    xaxis3=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    yaxis3=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    xaxis4=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    yaxis4=dict(
        linecolor='black',
        linewidth=2,
        mirror=True
    ),
    autosize=False,
    width=1050,
    height=900,
    title_text="Mutational scanning",
)

# Add annotations for R values in the top-left corner of each subplot
fig.add_annotation(xref="x1", yref="y1", x=-10, y=1.5, text=f"R = {r_value_epi_ESM:.2f}", showarrow=False, font=dict(size=12), row=1, col=1)
fig.add_annotation(xref="x2", yref="y2", x=-10, y=1.5, text=f"R = {r_value_epi:.2f}", showarrow=False, font=dict(size=12), row=1, col=2)
fig.add_annotation(xref="x3", yref="y3", x=-10, y=1.5, text=f"R = {r_value_inde:.2f}", showarrow=False, font=dict(size=12), row=2, col=1)
fig.add_annotation(xref="x4", yref="y4", x=-15, y=1.5, text=f"R = {r_value_ESM:.2f}", showarrow=False, font=dict(size=12), row=2, col=2)

fig.show()

# Print correlation values
print('R for Epistatic vs Experimental:', r_value_epi)
print('R for Independent vs Experimental:', r_value_inde)
print('R for ESM vs Experimental:', r_value_ESM)
print('R for Epistatic vs ESM:', r_value_epi_ESM)


R for Epistatic vs Experimental: 0.5065831203256604
R for Independent vs Experimental: 0.4796347167110549
R for ESM vs Experimental: 0.49795968387209866
R for Epistatic vs ESM: 0.8654054573690284


#### Get the average score of epistatic prediction from EVcouplings

In [36]:
import pandas as pd

# Read the CSV file into a DataFrame
# Replace 'your_data.csv' with the path to your actual CSV file
df = pd.read_csv('P450_D12_T86L_b0.7_single_mutant_matrix.csv')

# Group by 'pos' and calculate the mean of 'prediction_epistatic'
average_prediction = df.groupby('pos')['prediction_epistatic'].mean().reset_index()

# Convert to a list
average_prediction[['pos', 'prediction_epistatic']].to_csv('EV_epistatic_scores.txt', sep=' ', index=False, header=False)

# Sort the positions based on 'prediction_epistatic' in descending order
average_prediction_sorted = average_prediction.sort_values(by='prediction_epistatic', ascending=False)

# Output the sorted DataFrame to an Excel file
average_prediction.to_excel('average_prediction.xlsx', index=False)

# Output the sorted DataFrame to an Excel file
average_prediction_sorted.to_excel('average_prediction_sorted.xlsx', index=False)

# Display the result
print(average_prediction)


     pos  prediction_epistatic
0     15             -8.368183
1     16             -6.098008
2     17             -7.480300
3     18             -8.514412
4     19             -5.110641
..   ...                   ...
364  393             -5.621736
365  394             -8.289089
366  395             -7.168160
367  396             -8.980749
368  397             -6.001109

[369 rows x 2 columns]
